In [ ]:
import cv2
import os
from google.colab import drive
drive.mount('/content/drive')

def load_and_extract_frames(video_path, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:  # End of video
            break

        frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved {frame_filename}")
        frame_count += 1

    cap.release()
    print(f"Total frames extracted: {frame_count}")

video_path = "/content/drive/MyDrive/Dataset/sample.mp4"
output_dir = "/content/drive/MyDrive/Dataset/ExtractedFrames"
load_and_extract_frames(video_path, output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0000.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0001.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0002.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0003.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0004.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0005.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0006.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0007.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0008.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0009.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0010.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/frame_0011.jpg
Saved /content/drive/MyDrive/Dataset/ExtractedFrames/fr

In [12]:
import numpy as np

def segment_car(frame):

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_orange = np.array([10, 100, 100])
    upper_orange = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower_orange, upper_orange)
    segmented_frame = cv2.bitwise_and(frame, frame, mask=mask)

    return segmented_frame, mask

def track_car(frames):
    tracked_frames = []
    previous_centroids = []

    for frame in frames:
        segmented_frame, mask = segment_car(frame)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        current_centroids = []

        for contour in contours:
            if cv2.contourArea(contour) > 300:
                x, y, w, h = cv2.boundingRect(contour)
                cX, cY = x + w // 2, y + h // 2
                current_centroids.append((cX, cY))
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)

        tracked_frames.append(frame)
        previous_centroids = current_centroids

    return tracked_frames

def foreground_background_segmentation(frames):

    fg_frames = []
    background_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

    for frame in frames:
        fg_mask = background_subtractor.apply(frame)
        fg_mask = cv2.medianBlur(fg_mask, 5)
        fg_frame = cv2.bitwise_and(frame, frame, mask=fg_mask)
        fg_frames.append(fg_frame)

    return fg_frames

def process_car_frames(input_dir, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])
    frames = [cv2.imread(os.path.join(input_dir, file)) for file in frame_files]

    segmented_frames = [segment_car(frame)[0] for frame in frames]
    tracked_frames = track_car(frames)
    fg_frames = foreground_background_segmentation(frames)

    for idx, (seg, tr, fg) in enumerate(zip(segmented_frames, tracked_frames, fg_frames)):
        cv2.imwrite(os.path.join(output_dir, f"segmented_{idx:04d}.jpg"), seg)
        cv2.imwrite(os.path.join(output_dir, f"tracked_{idx:04d}.jpg"), tr)
        cv2.imwrite(os.path.join(output_dir, f"foreground_{idx:04d}.jpg"), fg)

    print("Processing completed. Results saved to:", output_dir)

input_dir = "/content/drive/MyDrive/Dataset/ExtractedFrames"
output_dir = "/content/drive/MyDrive/Dataset/CarTrackingResults"
process_car_frames(input_dir, output_dir)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing completed. Results saved to: /content/drive/MyDrive/Dataset/CarTrackingResults


In [14]:
from skimage.metrics import structural_similarity as ssim

def histogram_similarity(frame1, frame2):

    hist1 = cv2.calcHist([frame1], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([frame2], [0, 1, 2], None, [256, 256, 256], [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)
    correlation = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)
    return correlation

def pixel_diff(frame1, frame2):
    diff = cv2.absdiff(frame1, frame2)
    mse = np.sum(diff ** 2) / float(frame1.shape[0] * frame1.shape[1])
    return mse

def detect_scene_cuts(frames, threshold_hard_cut=0.7, threshold_soft_cut=3000):

    scene_cuts = []
    for i in range(1, len(frames)):
        frame1 = frames[i-1]
        frame2 = frames[i]
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        hist_corr = histogram_similarity(frame1, frame2)
        mse = pixel_diff(gray1, gray2)
        if hist_corr < threshold_hard_cut:
            scene_cuts.append((i, 'hard'))
        elif mse > threshold_soft_cut:
            scene_cuts.append((i, 'soft'))

    return scene_cuts

def visualize_scene_cuts(frames, scene_cuts, output_dir):

    os.makedirs(output_dir, exist_ok=True)

    for idx, cut_type in scene_cuts:
        frame = frames[idx]
        cut_type_text = f"{cut_type} cut"
        cv2.putText(frame, cut_type_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        cv2.rectangle(frame, (50, 100), (400, 200), (0, 0, 255), 2)
        cv2.imwrite(os.path.join(output_dir, f"scene_cut_{idx:04d}.jpg"), frame)
    print(f"Scene cuts visualized and saved to: {output_dir}")


def process_scene_cuts(input_dir, output_dir):

    os.makedirs(output_dir, exist_ok=True)
    frame_files = sorted([f for f in os.listdir(input_dir) if f.endswith('.jpg')])
    frames = [cv2.imread(os.path.join(input_dir, file)) for file in frame_files]
    scene_cuts = detect_scene_cuts(frames)
    visualize_scene_cuts(frames, scene_cuts, output_dir)
    print("Detected Scene Cuts:")
    for cut in scene_cuts:
        print(f"Frame {cut[0]}: {cut[1]}")

input_dir = "/content/drive/MyDrive/Dataset/ExtractedFrames"
output_dir = "/content/drive/MyDrive/Dataset/SceneCutsResults"
process_scene_cuts(input_dir, output_dir)


Scene cuts visualized and saved to: /content/drive/MyDrive/Dataset/SceneCutsResults
Detected Scene Cuts:
